# Problem Description

Describe how you could use a single array to implement three stacks.

# Solution

Implementing three stacks using a single array can be challenging, but it is achievable with two approaches: Fixed Division and Flexible Division.

### Fixed Division

In this approach, the array is divided into three fixed segments, one for each stack.

In [1]:
class FixedMultiStack:
    def __init__(self, stackSize) -> None:
        self.numberOfStacks: int = 3
        self.stackCapacity: int = stackSize
        self.values: list[int] = [0] * (self.numberOfStacks * stackSize)
        self.sizes: list[int] = [0] * self.numberOfStacks

    def push(self, stackNum: int, value: int) -> None:
        if self.isFull(stackNum):
            return
        
        self.sizes[stackNum] += 1
        self.values[self.indexOfTop(stackNum)] = value

    def pop(self, stackNum: int) -> int:
        if self.isEmpty(stackNum):
            return
        
        topIndex: int = self.indexOfTop(stackNum)
        value: int = self.values[topIndex]
        self.values[topIndex] = 0
        self.sizes[stackNum] -= 1
        return value

    def peek(self, stackNum: int) -> int:
        if self.isEmpty(stackNum):
            return
        
        topIndex: int = self.indexOfTop(stackNum)
        return self.values[topIndex]

    def isEmpty(self, stackNum: int) -> bool:
        return self.sizes[stackNum] == 0

    def isFull(self, stackNum: int) -> bool:
        return self.sizes[stackNum] == self.stackCapacity

    def indexOfTop(self, stackNum: int) -> int:
        offset: int = stackNum * self.stackCapacity
        size: int = self.sizes[stackNum]
        return offset + size - 1

### Flexible Division

In this approach, the stacks can grow and shrink dynamically by shifting elements.

In [2]:
class Solution:
    values:list = []

    class StackInfo:
        def __init__(self, start, capacity) -> None:
            self.start: int = start
            self.size: int = 0
            self.capacity: int = capacity
        
        def isWithinStackCapacity(self, index) -> bool:
            if index < 0 or index >= len(Solution.values):
                return False
            contiguousIndex: int = index + len(Solution.values) if index < self.start else index
            end: int = self.start + self.capacity
            return self.start <= contiguousIndex and contiguousIndex < end

        def lastCapacityIndex(self) -> int:
            return self.adjustIndex(self.start + self.capacity - 1)

        def lastElementIndex(self) -> int:
            return self.adjustIndex(self.start + self.size - 1)
        
        def isFull(self) -> bool:
            return self.size == self.capacity

        def isEmpty(self) -> bool:
            return self.size == 0
        
        def adjustIndex(self, index: int) -> int:
            max: int = len(Solution.values)
            return ((index % max) + max) % max

        def nextIndex(self, index: int) -> int:
            return self.adjustIndex(index + 1)
        
        def previousIndex(self, index: int) -> int:
            return self.adjustIndex(index - 1)

    class FlexibleMultiStack:
        def __init__(self, numberOfStacks: int, defaultSize: int) -> None:
            self.info: list[Solution.StackInfo] = [Solution.StackInfo(defaultSize * i, defaultSize) for i in range(numberOfStacks)]
            Solution.values = [0] * (numberOfStacks * defaultSize)

        def push(self, stackNum: int, value: int) -> None:
            if self.allStacksAreFull():
                return
            
            stack = self.info[stackNum]
            if stack.isFull():
                self.expand(stackNum)

            stack.size += 1
            Solution.values[stack.lastElementIndex()] = value
        
        def pop(self, stackNum: int) -> int:
            stack = self.info[stackNum]
            if stack.isEmpty():
                return
            
            value: int = Solution.values[stack.lastElementIndex()]
            Solution.values[stack.lastElementIndex()] = 0
            stack.size -= 1
            return value
    
        def peek(self, stackNum: int) -> int:
            stack = self.info[stackNum]
            if stack.isEmpty():
                return
            
            return Solution.values[stack.lastElementIndex()]

        def shift(self, stackNum: int) -> None:
            stack = self.info[stackNum]
            if stack.size >= stack.capacity:
                nextStack: int = (stackNum + 1) % len(self.info)
                self.shift(nextStack)
                stack.capacity += 1
            
            index: int = stack.lastCapacityIndex()
            while stack.isWithinStackCapacity(index):
                Solution.values[index] = Solution.values[stack.previousIndex(index)]
                index = stack.previousIndex(index)
            
            Solution.values[stack.start] = 0
            stack.start = stack.nextIndex(stack.start)
            stack.capacity -= 1

        def expand(self, stackNum: int) -> None:
            self.shift((stackNum + 1) % len(self.info))
            self.info[stackNum].capacity += 1
        
        def numberOfElements(self) -> int:
            size: int = 0
            for sd in self.info:
                size += sd.size
            return size
        
        def allStacksAreFull(self) -> bool:
            return self.numberOfElements() == len(Solution.values)

## Explanation
1. Fixed Division:
   * Divides the array into three fixed segments.
   * Each segment operates as an independent stack.
   * This approach is simple but may lead to inefficient use of space.
2. Flexible Division:
   * Allows the stacks to grow and shrink dynamically.
   * Shifts elements in the array to accommodate growth.
   * This approach is more complex but ensures efficient use of space.

Both approaches have their advantages and can be chosen based on the specific requirements of the problem.

## Example Usage
These examples demonstrate how to use the FixedMultiStack and FlexibleMultiStack classes to implement three stacks using a single array.

### FixedMultiStack

In [3]:
# Create a FixedMultiStack with a stack size of 4 for each stack
fixedStack = FixedMultiStack(4)

# Push elements onto each stack
fixedStack.push(0, 10)  # Push 10 onto stack 0
fixedStack.push(1, 20)  # Push 20 onto stack 1
fixedStack.push(2, 30)  # Push 30 onto stack 2

# Push more elements onto stack 0
fixedStack.push(0, 11)
fixedStack.push(0, 12)

# Pop elements from each stack
print(fixedStack.pop(0))  # Output: 12
print(fixedStack.pop(1))  # Output: 20
print(fixedStack.pop(2))  # Output: 30

# Peek at the top element of stack 0
print(fixedStack.peek(0))  # Output: 11

# Check if a stack is empty
print(fixedStack.isEmpty(0))  # Output: False
print(fixedStack.isEmpty(1))  # Output: True

12
20
30
11
False
True


### FlexibleMultiStack

In [5]:
# Create a FlexibleMultiStack with 3 stacks and a default size of 4
flexibleStack = Solution.FlexibleMultiStack(3, 4)

# Push elements onto each stack
flexibleStack.push(0, 10)  # Push 10 onto stack 0
flexibleStack.push(1, 20)  # Push 20 onto stack 1
flexibleStack.push(2, 30)  # Push 30 onto stack 2

# Push more elements onto stack 0
flexibleStack.push(0, 11)
flexibleStack.push(0, 12)
flexibleStack.push(0, 13)
flexibleStack.push(0, 14)  # This may trigger an expansion if stack 0 is full

# Pop elements from each stack
print(flexibleStack.pop(0))  # Output: 14
print(flexibleStack.pop(1))  # Output: 20
print(flexibleStack.pop(2))  # Output: 30

# Peek at the top element of stack 0
print(flexibleStack.peek(0))  # Output: 13

14
20
30
13


# Literature

The contents base on the following literature:

* Gayle Laakmann McDowell, *Cracking the Coding Interview*, [Link](https://www.crackingthecodinginterview.com/).

**Copyright**

The notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebooks for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT).